In [1]:
TESTING_SUBSET = False  # ***** FOR TESTING ONLY !!!!!!!! ******

import sys
IN_COLAB = 'google.colab' in sys.modules

In [2]:
import datetime
import numpy as np
from sklearn.metrics import fbeta_score
from matplotlib import pyplot as plt
import math
import pandas as pd
import sys
import json
import time
import torch
from torch import nn
import random
#from tqdm.auto import tqdm
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import Dataset
import os
import torchsummary
from itertools import product
from transformers import AutoTokenizer, AutoModel
from dataclasses import dataclass
from transformers import PreTrainedTokenizer
from torch.utils.data import DataLoader

if IN_COLAB:
  sys.path.append('/content/drive/MyDrive/ML/lecr/code/')
from l_utils import *
from model_utils import *
from fe_utils import *

#from model_defs import Topic_NN, Content_NN

%env CUBLAS_WORKSPACE_CONFIG=:16:8

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


/home/mahesh/Desktop/ML/kaggle/LearningEquality/learning-equality-code/le-venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-12 13:54:14.207969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-04-12 13:54:14.352779: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2023-04-12 13:54:14.815257: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-04-12 13:54:14.815322: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-04-12 13:54:14.815328: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


env: CUBLAS_WORKSPACE_CONFIG=:16:8
cuda


In [3]:
if IN_COLAB:
    class CFG:
        MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        DIRNAME_PROCESSED_DATA = '/content/drive/MyDrive/ML/lecr/data/'
        FILENAME_DATA_CONTENT_F = DIRNAME_PROCESSED_DATA + "content.f"
        FILENAME_DATA_TOPICS_F = DIRNAME_PROCESSED_DATA + "topics.f"
        FILENAME_DATA_CORRELATIONS_CSV = DIRNAME_PROCESSED_DATA + "correlations.csv"
        NUM_FOLDS = 5
        SAVE_MODEL_DIR = DIRNAME_PROCESSED_DATA + "/models/"
else:
    from cfg import *
topics_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "topics_tokenized_for_ranker.pq"
contents_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "contents_tokenized_for_ranker.pq"

In [4]:
auto_tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL)

In [5]:
contents_df = pd.read_feather(CFG.FILENAME_DATA_CONTENT_F)
topics_df   = pd.read_feather(CFG.FILENAME_DATA_TOPICS_F)
correlations_df =  pd.read_csv(CFG.FILENAME_DATA_CORRELATIONS_CSV)

In [6]:
topics_tokenized_df, contents_tokenized_df = ranker_tokenize_main(auto_tokenizer, topics_df.copy(),  contents_df.copy(), max_length = 64)

Map (num_proc=2):   0% 0/61517 [00:00<?, ? examples/s]

Map (num_proc=2):   2% 1000/61517 [00:00<00:45, 1328.92 examples/s]

Map (num_proc=2):   5% 3000/61517 [00:00<00:14, 3931.71 examples/s]

Map (num_proc=2):   8% 5000/61517 [00:01<00:09, 6009.38 examples/s]

Map (num_proc=2):  11% 7000/61517 [00:01<00:07, 7632.99 examples/s]

...

Map (num_proc=2):  97% 150000/154047 [04:14<00:05, 696.08 examples/s]

Map (num_proc=2):  98% 151000/154047 [04:17<00:05, 523.09 examples/s]

Map (num_proc=2):  99% 152000/154047 [04:17<00:02, 688.50 examples/s]

Map (num_proc=2):  99% 153024/154047 [04:20<00:01, 555.19 examples/s]

Map (num_proc=2): 100% 154024/154047 [04:23<00:00, 447.40 examples/s]

In [7]:
topics_tokenized_df.to_parquet(topics_tokenized_pq_file)
contents_tokenized_df.to_parquet(contents_tokenized_pq_file)